In [119]:
import pandas as pd
import numpy as np
from sklearn.utils import shuffle
from torch.autograd import Variable
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms, utils
import torch.nn.functional as F
import torch


In [115]:
df = pd.read_csv("datasets/breast-cancer-wisconsin.csv",sep=',')
df = shuffle(df)
train_df = df[:500]
test_df = df[500:]

In [116]:
def getDS(df):
    df1 = df.iloc[:,1:10]
    df2 = df.iloc[:,10]
    y = np.array(df2)
    y[y==4]=1 # malignant
    y[y==2]=0 # benign
    X = np.array(df1)
    X = X.astype(float)
    return X,y

In [117]:
train = getDS(train_df)
test = getDS(test_df)

In [118]:
class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.fc1 = nn.Linear(9, 50)
        self.fc2 = nn.Linear(50, 2)
        self.scores = nn.Softmax()

    def forward(self, x):
        x = F.relu(self.fc1(x))
        x = self.fc2(x)
        x = self.scores(x)
        return x

In [121]:
net = Net()
criterion = nn.NLLLoss()
optimizer = optim.SGD(net.parameters(), lr=0.001, momentum=0.9)

In [122]:

class CancerDataset(Dataset):
    """CancerDataset."""

    def __init__(self, X, y):
        """
        Args:
            csv_file (string): Path to the csv file with annotations.
            root_dir (string): Directory with all the images.
            transform (callable, optional): Optional transform to be applied
                on a sample.
        """
        self.X = X
        self.y = y

    def __len__(self):
        return (self.X.shape[0])

    def __getitem__(self, idx):
        sample = {'X': X[idx], 'classes': y[idx]}
        return sample

In [124]:
train_ds = CancerDataset(train[0],train[1])
test_ds = CancerDataset(test[0],test[1])

In [125]:
train_dataloader = DataLoader(train_ds, batch_size=20,shuffle=True, num_workers=4)
test_dataloader = DataLoader(test_ds, batch_size=20,shuffle=True, num_workers=4)

In [126]:
net.train()
for epoch in range(2):  # loop over the dataset multiple times

    running_loss = 0.0
    for i, data in enumerate(train_dataloader, 0):
        # get the inputs
        inputs, labels = data['X'], data['classes']
        # wrap them in Variable
        inputs, labels = Variable(inputs.float()), Variable(labels)

        # zero the parameter gradients
        optimizer.zero_grad()

        # forward + backward + optimize
        outputs = net(inputs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

#         # print statistics
        running_loss += loss.data[0]
        if i % 2000 == 1999:    # print every 2000 mini-batches
            print('[%d, %5d] loss: %.3f' %
                  (epoch + 1, i + 1, running_loss / 2000))
            running_loss = 0.0

print('Finished Training')


Finished Training


In [127]:
net.eval()
test_loss = 0
correct = 0
total = 0
for batch_idx, (inputs, targets) in enumerate(test_dataloader):
    inputs, labels = data['X'], data['classes']
    inputs, targets = Variable(inputs.float()), Variable(labels)
    outputs = net(inputs)
    loss = criterion(outputs, targets)

    test_loss += loss.data[0]
#     print outputs.data
    _, predicted = torch.max(outputs.data, 1)
#     print predicted
#     print targets.data
#     if(batch_idx == 1):
#         break
    total += targets.size(0)
    correct += predicted.eq(targets.data).cpu().sum()


In [135]:
print "test set Accuracy", correct/float(total)

test set Accuracy 0.9
